**Small Dataset**

In [ ]:
!git clone https://github.com/coick4698/garbage_classification.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/garbage_classification

import sys
sys.path.append('/content/garbage_classification/src')

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from dataset import get_dataloaders
from model import get_mobilenetv2, get_densenet121, get_squeezenet, get_shufflenetv2
from train import train_model
from utils import plot_accuracy, plot_confusion_matrix, eval_precision_recall_f1

data_path = '/content/drive/MyDrive/iss_project/garbage_classification'

In [ ]:
# DenseNet121 - Small dataset (5m)
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)

model = get_densenet121(num_classes=12).to(device)
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

plot_accuracy(train_acc, val_acc, title="DenseNet121 - Small Dataset")
plot_confusion_matrix(model, val_loader, class_names, device=device)
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

torch.save(model.state_dict(), "results/checkpoints/densenet121_small_epoch10.pth")

In [ ]:
# MobilenetV2 - Small dataset (2m)
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)

model = get_mobilenetv2(num_classes=12).to(device)
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

plot_accuracy(train_acc, val_acc, title="MobileNetV2 - Small Dataset")
plot_confusion_matrix(model, val_loader, class_names, device=device)
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

torch.save(model.state_dict(), "results/checkpoints/mobilenetv2_small_epoch10.pth")

In [ ]:
# Squeezenet - Small dataset(2m)
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)

model = get_squeezenet(num_classes=12).to(device)
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

plot_accuracy(train_acc, val_acc, title="SqueezeNet - Small Dataset")
plot_confusion_matrix(model, val_loader, class_names, device=device)
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

torch.save(model.state_dict(), "results/checkpoints/squeezenet_small_epoch10.pth")

In [ ]:
# ShufflenetV2 - Small dataset (1m)
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)

model = get_shufflenetv2(num_classes=12).to(device)
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

plot_accuracy(train_acc, val_acc, title="ShuffleNetV2 - Small Dataset")
plot_confusion_matrix(model, val_loader, class_names, device=device)
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

torch.save(model.state_dict(), "results/checkpoints/shufflenetv2_small_epoch10.pth")

In [ ]:
# Exp.1.5 - FC layer + Dropout + Activation Function
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2
import matplotlib.pyplot as plt
import cv2
import os

# 1. Define model
class CustomMobileNetV2_exp1(nn.Module):
    def __init__(self, num_classes=12):
        super(CustomMobileNetV2_exp1, self).__init__()
        base_model = mobilenet_v2(weights='IMAGENET1K_V1')
        self.features = base_model.features
        self.pool = nn.AdaptiveAvgPool2d(1)

        # FC layer: 1280 -> 256 -> 12
        # Dropout(0.2,0.3)
        # Activation Function(ReLU6 -> GELU)
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)

        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 2. load data
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. load model
model = CustomMobileNetV2_exp1(num_classes=12).to(device)
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

# 4. Accuracy Plot
plot_accuracy(train_acc, val_acc, title="Custom MobileNetV2-Exp1")

# 5. Confusion Matrix
plot_confusion_matrix(model, val_loader, class_names, device=device)

# 6. Precision / Recall / F1-score
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

# 7. Save
torch.save(model.state_dict(), "results/checkpoints/custommobilenetv2_exp1_small_epoch10.pth")

In [ ]:
# Exp.2.5 - Transfer Learning Fine-tuning (partial) - Small dataset

import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

# 1. Define model (pretrained + custom head)
class CustomMobileNetV2_exp2(nn.Module):
    def __init__(self, num_classes=12):
        super().__init__()
        base = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
        self.features = base.features
        self.pool = nn.AdaptiveAvgPool2d(1)

        # FC layer: 1280 -> 256 -> 12
        # Dropout(0.2,0.3)
        # Activation Function(ReLU6 -> GELU)
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 2. utilize boundary of fine-tuning
def set_finetune_mode(model: CustomMobileNetV2_exp2, mode="partial", unfreeze_last_blocks=2):
    """
    mode: 'freeze' | 'partial' | 'full'
    - freeze  : Freeze the entire backbone (use as feature extractor, train head only)
    - partial : Train only the last N blocks of the backbone
    - full    : Train the entire backbone
    """
    for p in model.features.parameters():
        p.requires_grad = False

    if mode == "full":
        for p in model.features.parameters():
            p.requires_grad = True
    elif mode == "partial":
        blocks = list(model.features.children())
        for m in blocks[-unfreeze_last_blocks:]:
            for p in m.parameters():
                p.requires_grad = True
    elif mode == "freeze":
        pass
    else:
        raise ValueError("mode should be one of {'freeze','partial','full'}")

# 3. load data
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 4. load model
model = CustomMobileNetV2_exp2(num_classes=12).to(device)

# 5. select fine-tuning mode
finetune_mode = "partial"                # Freeze the entire backbone (use as feature extractor, train head only)
set_finetune_mode(model, mode=finetune_mode, unfreeze_last_blocks=2)

# 6. Training
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

# 7. Accuracy Plot
plot_accuracy(train_acc, val_acc, title="Custom MobileNetV2-Exp2")

# 8. Confusion Matrix
plot_confusion_matrix(model, val_loader, class_names, device=device)

# 9. Precision / Recall / F1-score
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

# 10. Save
torch.save(model.state_dict(), "results/checkpoints/custommobilenetv2_exp2_small_epoch10.pth")

In [ ]:
# Exp.3.5 - Transfer Learning Fine-tuning (full) - Small dataset

import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

# 1. Define model (pretrained + custom head)
class CustomMobileNetV2_exp3(nn.Module):
    def __init__(self, num_classes=12):
        super().__init__()
        base = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
        self.features = base.features
        self.pool = nn.AdaptiveAvgPool2d(1)

        # FC layer: 1280 -> 256 -> 12
        # Dropout(0.2,0.3)
        # Activation Function(ReLU6 -> GELU)
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 2. utilize boundary of fine-tuning
def set_finetune_mode(model: CustomMobileNetV2_exp3, mode="partial", unfreeze_last_blocks=2):
    """
    mode: 'freeze' | 'partial' | 'full'
    - freeze  : Freeze the entire backbone (use as feature extractor, train head only)
    - partial : Train only the last N blocks of the backbone
    - full    : Train the entire backbone
    """
    for p in model.features.parameters():
        p.requires_grad = False

    if mode == "full":
        for p in model.features.parameters():
            p.requires_grad = True
    elif mode == "partial":
        blocks = list(model.features.children())
        for m in blocks[-unfreeze_last_blocks:]:
            for p in m.parameters():
                p.requires_grad = True
    elif mode == "freeze":
        pass
    else:
        raise ValueError("mode should be one of {'freeze','partial','full'}")

# 3. load data
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 4. load model
model = CustomMobileNetV2_exp3(num_classes=12).to(device)

# 5. select fine-tuning mode
finetune_mode = "full"                # Train the entire backbone
set_finetune_mode(model, mode=finetune_mode, unfreeze_last_blocks=2)

# 6. Training
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

# 7. Accuracy Plot
plot_accuracy(train_acc, val_acc, title="Custom MobileNetV2-Exp3")

# 8. Confusion Matrix
plot_confusion_matrix(model, val_loader, class_names, device=device)

# 9. Precision / Recall / F1-score
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

# 10. Save
torch.save(model.state_dict(), "results/checkpoints/custommobilenetv2_exp3_small_epoch10.pth")

In [ ]:
# Exp.4.5 - Transfer Learning Fine-tuning (freeze) - Small dataset

import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

# 1. Define model (pretrained + custom head)
class CustomMobileNetV2_exp4(nn.Module):
    def __init__(self, num_classes=12):
        super().__init__()
        base = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
        self.features = base.features
        self.pool = nn.AdaptiveAvgPool2d(1)

        # FC layer: 1280 -> 256 -> 12
        # Dropout(0.2,0.3)
        # Activation Function(ReLU6 -> GELU)
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 2. utilize boundary of fine-tuning
def set_finetune_mode(model: CustomMobileNetV2_exp4, mode="partial", unfreeze_last_blocks=2):
    """
    mode: 'freeze' | 'partial' | 'full'
    - freeze  : Freeze the entire backbone (use as feature extractor, train head only)
    - partial : Train only the last N blocks of the backbone
    - full    : Train the entire backbone
    """
    for p in model.features.parameters():
        p.requires_grad = False

    if mode == "full":
        for p in model.features.parameters():
            p.requires_grad = True
    elif mode == "partial":
        blocks = list(model.features.children())
        for m in blocks[-unfreeze_last_blocks:]:
            for p in m.parameters():
                p.requires_grad = True
    elif mode == "freeze":
        pass
    else:
        raise ValueError("mode should be one of {'freeze','partial','full'}")

# 3. load data
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 4. load model
model = CustomMobileNetV2_exp4(num_classes=12).to(device)

# 5. select fine-tuning mode
finetune_mode = "freeze"                # Freeze the entire backbone (use as feature extractor, train head only)
set_finetune_mode(model, mode=finetune_mode, unfreeze_last_blocks=2)

# 6. Training
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

# 7. Accuracy Plot
plot_accuracy(train_acc, val_acc, title="Custom MobileNetV2-Exp4")

# 8. Confusion Matrix
plot_confusion_matrix(model, val_loader, class_names, device=device)

# 9. Precision / Recall / F1-score
acc, report, y_true, y_pred = eval_precision_recall_f1(model, val_loader, class_names, device)

# 10. Save
torch.save(model.state_dict(), "results/checkpoints/custommobilenetv2_exp4_small_epoch10.pth")